# Image filtering

### Pull data

In [ ]:
"we import (=clone) all the data or just update (=pull) them"
import os

if not os.path.exists("assets_signal"):
    print("the directory assets_signal is created")
    !git clone https://github.com/vincentvigon/assets_signal
else:
    print("the directory assets_signal is updated")
    %cd assets_signal
    !git pull https://github.com/vincentvigon/assets_signal
    %cd ..

### Import

In [ ]:
%reset -f

In [ ]:
import numpy as np
import scipy
from scipy import signal as sg
from scipy import ndimage
import  matplotlib.pyplot as plt

import imageio
import numpy
import matplotlib.pyplot as plt
import numpy as np
plt.style.use("default")
import IPython

np.set_printoptions(linewidth=500,suppress=True,precision=5)

### One image

We import an image and keep only one channel. Data are converted as floats and standardized. So pixels range in $[0,1]$.

In [ ]:
img_origin = imageio.imread("assets_signal/babouin/babouin_moyen.jpg")[:,:,0].astype(np.float32)
img_origin/=255.
img_origin.shape,img_origin.dtype,np.max(img_origin)

## Convolution

### Convolution by ourself

The discrete convolution between an image (with one channel) and a mask (=a small matrix) is:
$$
\Big(image \star mask\Big) [i,j]  = \sum_{di,dj} image[i+di,j+dj] \, mask[di, dj]
$$
It is a sort of moving average along the image, ponderated by the mask.


***To you:*** $(1\heartsuit)$ Do you see some difference with the 'mathematical' convolution?


Here is a natural implementation using 4 loops: 2 explicits (for-loop) and 2 implicit (`numpy` loop).

In [ ]:
def convolution2D(image, mask):

    assert mask.shape[0]%2==1 and mask.shape[1]%2==1, "we want mask with a pixel at the center"

    h,w=image.shape

    II,JJ=mask.shape
    I = (II-1) // 2
    J = (JJ-1) // 2

    """we add padding (=marge) to avoid side problems.
    One can also use np.pad() """
    img_pad = np.zeros([h+2*I,w+2*J])
    img_pad[I:-I,J:-J]=image

    res=np.zeros_like(image)

    for i in range(0,h):
        for j in range(0,w):
            res[i,j]=np.sum(img_pad[i:i+II , j:j+JJ] * mask)
    return res

In [ ]:
image=img_origin.copy()
II=21
JJ=3

mask = np.ones((II,JJ))/II/JJ
print(mask)

In [ ]:
%%time
imageConv = convolution2D(image,mask)

In [ ]:
plt.imshow(imageConv,cmap='gray');

***To you:***

* $(1\heartsuit)$ Explain the visual aspect of the image above.
* $(2\heartsuit)$ Find a mask of shape $5\times5$ that just shift the image.
*  $(2\heartsuit)$ Find a mask of shape $3\times3$ which allows to show the vertical variations of the image. If you have no idea, it will come during this chapter.

### With a rolling window

A natural way to implement the convolution is to use a rolling window.

In [ ]:
""" this function produce a rolling window which is just a view on the data:
= a special way to move in the data. No copy is made.
"""
def rolling_windows_img(a,kshape):
    outShape=(a.shape[0]-kshape[0]+1,)+(a.shape[1]-kshape[1]+1,)+kshape
    outStrides=a.strides+a.strides
    return np.lib.stride_tricks.as_strided(a,shape=outShape,strides=outStrides)

In [ ]:
rolling=rolling_windows_img(image,mask.shape)
rolling.shape

In [ ]:
pre_imageConv=rolling * mask[np.newaxis,np.newaxis,:,:]
pre_imageConv.shape

***To you:***

* $(2\heartsuit)$ Finish the job: remake the function `convolution2D` using the previous function `rolling_windows_img`. You can suppress the double loop `for i: for j:`
* $(2\heartsuit)$ Compare the performances with the first implementation. The creation of the rolling window must also be  counted. But, does it really take some time?

### Convolution by scipy

Now we use the `scipy` function. It is faster because all the 4 loops are implicit.

In [ ]:
%time
imageConv2 = sg.convolve(image,mask,mode="same")

In [ ]:
plt.imshow(imageConv2,cmap='gray');

Look at the docstring:   'mode'  can be :

        ``full``
           The output is the full discrete linear convolution
           of the inputs. (Default)
         => image size increases.
        
         
        ``valid``
           The output consists only of those elements that do not
           rely on the zero-padding. In 'valid' mode, either `in1` or `in2`
           must be at least as large as the other in every dimension.
        => image size decreases
        
        ``same``
           The output is the same size as `in1`, centered
           with respect to the 'full' output.
           => image size stays the same
           
           
* The mode 'full' is not really interesting.
* The mode 'same' is pratical because we keep the same dimensions. But you can observe some side effects (effet de bord).
* If you use the mode 'valid', you loose a part of the image, but no side effects.  



***To you:***

* ($5\heartsuit\flat$) Modify the first convolution function to allow the 'valid' mode. The difficulty is to start the loops at the good index.
* ($2\heartsuit$) Find a formula that relies the shapes of the image, the shape of the mask and the shape of  the result, when we use the 'valid' mode.

## Smoothing

### Gaussian mask

A good mask to smooth an image is the gaussian mask: a discretisation of the gaussian density. This produice a moving-average, where the close pixels are more important than the far ones.

In [ ]:
def gaussian_mask(shape=(3, 3), sigma=0.5):

    if len(shape)!=2: raise ValueError("len-2 shape required")
    if shape[0]%2!=1 or shape[1]%2!=1: raise ValueError("only odd sizes are OK")

    m,n = (shape[0]-1)/2,(shape[1]-1)/2
    y,x = np.ogrid[-m:m+1,-n:n+1]
    res = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )


    """normalization"""
    res/=res.sum()
    return res

In [ ]:
print(gaussian_mask((11, 11)))

In [ ]:
plt.imshow(np.log(gaussian_mask((11, 11))),cmap="gray");

Let's soften a gaussian noise with a gaussian mask:

In [ ]:
image=img_origin.copy()

"""random noise (with gaussian distribution)"""
image+=np.random.normal(scale=0.1,size=image.shape)
masque = gaussian_mask(shape=(5,5),sigma=2.)
imageConv=sg.convolve(image,masque, "valid")

fig,(ax0,ax1,ax2)=plt.subplots(1,3,figsize=(12,3))
ax0.imshow(img_origin, cmap='gray')
ax1.imshow(image,cmap='gray')
ax2.imshow(imageConv, cmap='gray');

***To you:*** $(1\heartsuit)$ Which sort of noise can be soften with a Gaussian mask?

### Impultional noise and median filter

An impultional noise is a strong perturbation, but which appears only at rare places.

In [ ]:
image=img_origin.copy()
proba=0.1
"""some pixels are modified (ex: parasite occuring during the transmission of the image)"""
image[np.random.uniform(size=image.shape)<proba]=0
plt.imshow(image,cmap="gray");

The median filter is perfect to suppress such a noise: You move a rolling window of a given size (ex: $3\times 3$). For each window,  you replace the pixel at the middle  by the median of the pixels of the window. Such a filter is present at two places in `scipy`:

    scipy.signal.medfilt
    scipy.ndimage.filters.median_filter

In [ ]:
image_improved=scipy.signal.medfilt(image,kernel_size=3)
plt.imshow(image_improved,cmap="gray");

***To you:*** $(2\heartsuit)$  some pixel are still black. How it is possible.  Rule the filter to suppress them, but, what do you see then...

***To you:*** $(1\heartsuit)$ A filter is said linear when the application img$\to$Filter(img) is linear. Which filters we saw before is linear?

### A revolutionnary filter

Your chef has an idea (naturaly, a brilliant idea) for a new denoising filter which combine smoothing and median. He explains you this idea on the phone:

<< Firstly, take a rolling window of size $5\times5$. Secondly, for each window, sort the pixels. Thirdly ... >>

Heck! The phone stops. No need to re-contact your chef. Create this revolutionnary filter by yourself $(8\heartsuit\flat)$.

